In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#забираем из Google Drive если есть)))
!cp ./drive/MyDrive/colab1/dlgo.zip ./dlgo.zip

In [3]:
#распаковка модулей DL-Go
!unzip -o dlgo.zip

Archive:  dlgo.zip
   creating: dlgo/
   creating: dlgo/agent/
  inflating: dlgo/agent/alphago.py   
  inflating: dlgo/agent/base.py      
  inflating: dlgo/agent/helpers.py   
  inflating: dlgo/agent/naive.py     
  inflating: dlgo/agent/pg.py        
  inflating: dlgo/agent/predict.py   
  inflating: dlgo/agent/termination.py  
  inflating: dlgo/agent/__init__.py  
   creating: dlgo/agent/__pycache__/
  inflating: dlgo/agent/__pycache__/base.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/base.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/termination.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/terminati

In [4]:
from dlgo.goboard_fast import GameState, Player
from dlgo import scoring

#from dlgo.goboard_fast import Point
#from keras.layers import Activation, BatchNormalization
from keras.layers import Conv2D, Dense, Flatten, Input
from keras.models import Model
from dlgo import zero

In [5]:
def simulate_game(board_size, black_agent, black_collector, white_agent, white_collector):
    """Симуляция игры бота с самим собой."""
    print('Старт игры!')
    game = GameState.new_game(board_size)
    agents = {
        Player.black: black_agent,
        Player.white: white_agent,
    }
    black_collector.begin_episode()
    white_collector.begin_episode()
    while not game.is_over():
        next_move = agents[game.next_player].select_move(game)
        game = game.apply_move(next_move)

    game_result = scoring.compute_game_result(game)
    if game_result.winner == Player.black:
        black_collector.complete_episode(1)
        white_collector.complete_episode(-1)
    else:
        black_collector.complete_episode(-1)
        white_collector.complete_episode(1)

## Один цикл обучения с подкреплением

In [6]:
# Для достижения сверхчеловеческого уровня игры системе AlphaGo Zero
# потребовалось сыграть около 5 млн. игр с самой собой.

board_size = 9
encoder = zero.ZeroEncoder(board_size)

In [7]:
board_input = Input(shape=encoder.shape(), name='board_input')
pb = board_input
#создайте сеть с 4-мя сверточными слоями. Для создания сильного бота можно
#добавить множество дополнительных слоев
for i in range(4):
    pb = Conv2D(64, (3,3),
                padding='same',
                data_format='channels_first',
                activation='relu')(pb)

#добавьте в сеть выход функции политики
policy_conv = Conv2D(2, (1,1),
                     data_format='channels_first',
                     activation='relu')(pb)
policy_flat = Flatten()(policy_conv)
policy_output = Dense(encoder.num_moves(), activation='softmax')(policy_flat)

#добавьте в сеть выход функции ценности
value_conv = Conv2D(1, (1,1),
                    data_format='channels_first',
                    activation='relu')(pb)
value_flat = Flatten()(value_conv)
value_hidden = Dense(256, activation='relu')(value_flat)
value_output = Dense(1, activation='tanh')(value_hidden)

In [8]:
model = Model(
    inputs=[board_input],
    outputs=[policy_output, value_output])

In [9]:
#здесь мы используем 10 раундов на ход для ускорения работы демонстрационной
#версии. В ходе реального обучения потребляется гораздо большее их количество.
#Система AGZ использовала 1600.
black_agent = zero.ZeroAgent(model, encoder, rounds_per_move=10, c=2.0)
white_agent = zero.ZeroAgent(model, encoder, rounds_per_move=10, c=2.0)
c1 = zero.ZeroExperienceCollector()
c2 = zero.ZeroExperienceCollector()
black_agent.set_collector(c1)
white_agent.set_collector(c2)

In [10]:
%%time

#сгенерируйте 5-ть игр перед началом обучения. В ходе реального обучения
#вам потребуются партии, состоящие из тысяч игр.
for i in range(5):
    simulate_game(board_size, black_agent, c1, white_agent, c2)

Старт игры!
1/1 [==============================] - 9s 9s/step
max=> nodeMoves= 82  score_branch= <function ZeroAgent.select_branch.<locals>.score_branch at 0x7fcac27831c0>
1/1 [==============================] - 0s 34ms/step
max=> nodeMoves= 82  score_branch= <function ZeroAgent.select_branch.<locals>.score_branch at 0x7fcac27831c0>
1/1 [==============================] - 0s 37ms/step
max=> nodeMoves= 82  score_branch= <function ZeroAgent.select_branch.<locals>.score_branch at 0x7fca334eb880>
1/1 [==============================] - 0s 28ms/step
max=> nodeMoves= 82  score_branch= <function ZeroAgent.select_branch.<locals>.score_branch at 0x7fca334ebbe0>
1/1 [==============================] - 0s 25ms/step
max=> nodeMoves= 82  score_branch= <function ZeroAgent.select_branch.<locals>.score_branch at 0x7fca334ebbe0>
1/1 [==============================] - 0s 34ms/step
max=> nodeMoves= 82  score_branch= <function ZeroAgent.select_branch.<locals>.score_branch at 0x7fca334eb7f0>
1/1 [=============

ValueError: ignored

In [11]:
exp = zero.combine_experience([c1, c2])
black_agent.train(exp, 0.01, 2048)

AttributeError: ignored